In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from math import sqrt
import scipy
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import statsmodels.formula.api as sm
import copy
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms
import sklearn
from math import sqrt
from functools import reduce
import numpy as np
#import ee
#import geemap
import geopandas as gpd
import rioxarray as rxr
import rasterstats as rs
from rasterio.plot import show
import rasterio

%matplotlib inline

In [27]:
plt.rcParams["font.family"] = "Arial"

# IRRIGATED

In [28]:
# NDVI SUM Data
points = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/duration_seasonalCompositeNDVI_2017_SUM_JAS.gpkg'
points = gpd.read_file(points)
points = points.groupby(['Class','ADM3_EN']).mean().reset_index()
points = points.loc[points['Class']=='Irrigated Rice']
points = points.rename(columns=dict(zip(list(points.columns[2:20]), list(np.arange(2001, 2019, 1)))))
points['param']='ndvi'
ndvi = points.melt(id_vars=["Class", "ADM3_EN", "param"], var_name="Date", value_name='value')

# PCP
param_name = 'pcp'
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_seasonalComposite_'+param_name+'_JAS.gpkg'
params = gpd.read_file(params).groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Irrigated Rice']
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
pcp = params.melt(id_vars=["Class", "ADM3_EN", "param"], var_name="Date", value_name='value')

# LST DAY
param_name = 'LST_Day_1km'
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_seasonalComposite_'+param_name+'_JAS.gpkg'
params = gpd.read_file(params).groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Irrigated Rice']
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
lstday = params.melt(id_vars=["Class", "ADM3_EN", "param"], var_name="Date", value_name='value')

# LST NIGHT
param_name = 'LST_Night_1km'
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_seasonalComposite_'+param_name+'_JAS.gpkg'
params = gpd.read_file(params).groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Irrigated Rice']
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
lstnight = params.melt(id_vars=["Class", "ADM3_EN", "param"], var_name="Date", value_name='value')

# ET
param_name = 'et'
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_seasonalComposite_'+param_name+'_JAS.gpkg'
params = gpd.read_file(params).groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Irrigated Rice']
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
et = params.melt(id_vars=["Class", "ADM3_EN", "param"], var_name="Date", value_name='value')

# PET
param_name = 'pet'
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_seasonalComposite_'+param_name+'_JAS.gpkg'
params = gpd.read_file(params).groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Irrigated Rice']
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
pet = params.melt(id_vars=["Class", "ADM3_EN", "param"], var_name="Date", value_name='value')

# ELEVATION
param_name = 'elevation'
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_ch3_elevation.gpkg'
params = gpd.read_file(params).groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Irrigated Rice']
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
elevation = params.melt(id_vars=["Class", "ADM3_EN", "param"], var_name="Date", value_name='value')
elevation['Date'] = 2017

# SLOPE
param_name = 'slope'
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_ch3_slope.gpkg'
params = gpd.read_file(params).groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Irrigated Rice']
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
slope = params.melt(id_vars=["Class", "ADM3_EN", "param"], var_name="Date", value_name='value')
slope['Date'] = 2017


In [29]:
# Subset temporally and pivot to table for linear regression
startYear = 2012
endYear = 2017
merged = pd.concat([ndvi, pcp, lstday, lstnight, et, pet, elevation, slope])
subbed = merged[(merged.Date <= endYear) & (merged.Date >= startYear)].groupby(['ADM3_EN','param']).mean().reset_index(drop=False)

df = subbed.pivot(index="ADM3_EN", columns=["param"],values="value").reset_index(drop=False)
df

param,ADM3_EN,LST_Day_1km,LST_Night_1km,elevation,et,ndvi,pcp,pet,slope
0,Anilao,29.752275,22.879352,25.000000,24.428704,2.161667,1021.584401,35.135185,0.679341
1,Banate,29.167690,23.270684,9.000000,25.580741,2.214025,897.541770,37.108333,0.464795
2,Barotac Nuevo,30.485585,23.150976,14.500000,25.610251,2.323344,1009.738954,36.803858,0.429941
3,Barotac Viejo,30.307110,23.119553,21.476190,24.637728,2.184279,907.282328,36.262727,0.973596
4,Dingle,30.343313,22.812495,23.000000,26.404654,2.259961,1042.818827,36.141766,0.656537
5,Dumangas,30.073989,23.647222,7.947368,27.070961,2.309406,1017.942077,37.917145,0.336499
6,Iloilo City,30.940884,23.578897,9.833333,22.376042,2.197569,1154.839328,32.598457,0.280021
7,Janiuay,30.734093,22.720741,44.000000,26.976389,2.278850,1069.234975,34.770833,0.000000
8,Leganes,30.171614,23.332358,11.384615,26.634028,2.290865,1106.655612,37.770274,0.235064
9,Mina,30.530469,23.080012,32.666667,28.689950,2.348275,1066.056944,38.955594,0.405078


In [37]:
formula = 'ndvi ~ LST_Day_1km + pet'
results = sm.ols(formula = formula, 
                 missing='drop',
                 data=df).fit()
print(results.summary())
print(pd.DataFrame(results.params).round(decimals=4).transpose())

                            OLS Regression Results                            
Dep. Variable:                   ndvi   R-squared:                       0.660
Model:                            OLS   Adj. R-squared:                  0.615
Method:                 Least Squares   F-statistic:                     14.59
Date:                Wed, 15 Dec 2021   Prob (F-statistic):           0.000303
Time:                        19:08:48   Log-Likelihood:                 34.353
No. Observations:                  18   AIC:                            -62.71
Df Residuals:                      15   BIC:                            -60.03
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -0.5806      0.667     -0.871      

/Users/delatorredm/miniconda3/envs/ee/lib/python3.8/site-packages/scipy/stats/stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


# RAINFED

In [6]:
# NDVI SUM Data
points = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/duration_seasonalCompositeNDVI_2017_SUM_JFM.gpkg'
points = gpd.read_file(points)
points = points.groupby(['Class','ADM3_EN']).mean().reset_index()
points = points.loc[points['Class']=='Rain-fed Rice']
points = points.rename(columns=dict(zip(list(points.columns[2:20]), list(np.arange(2001, 2019, 1)))))
points['param']='ndvi'
ndvi = points.melt(id_vars=["Class", "ADM3_EN", "param"], var_name="Date", value_name='value')

# PCP
param_name = 'pcp'
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_seasonalComposite_'+param_name+'_JFM.gpkg'
params = gpd.read_file(params).groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Rain-fed Rice']
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
pcp = params.melt(id_vars=["Class", "ADM3_EN", "param"], var_name="Date", value_name='value')

# LST DAY
param_name = 'LST_Day_1km'
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_seasonalComposite_'+param_name+'_JFM.gpkg'
params = gpd.read_file(params).groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Rain-fed Rice']
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
lstday = params.melt(id_vars=["Class", "ADM3_EN", "param"], var_name="Date", value_name='value')

# LST NIGHT
param_name = 'LST_Night_1km'
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_seasonalComposite_'+param_name+'_JFM.gpkg'
params = gpd.read_file(params).groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Rain-fed Rice']
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
lstnight = params.melt(id_vars=["Class", "ADM3_EN", "param"], var_name="Date", value_name='value')

# ET
param_name = 'et'
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_seasonalComposite_'+param_name+'_JFM.gpkg'
params = gpd.read_file(params).groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Rain-fed Rice']
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
et = params.melt(id_vars=["Class", "ADM3_EN", "param"], var_name="Date", value_name='value')

# PET
param_name = 'pet'
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_seasonalComposite_'+param_name+'_JFM.gpkg'
params = gpd.read_file(params).groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Rain-fed Rice']
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
pet = params.melt(id_vars=["Class", "ADM3_EN", "param"], var_name="Date", value_name='value')

# ELEVATION
param_name = 'elevation'
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_ch3_elevation.gpkg'
params = gpd.read_file(params).groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Rain-fed Rice']
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
elevation = params.melt(id_vars=["Class", "ADM3_EN", "param"], var_name="Date", value_name='value')
elevation['Date'] = 2017

# SLOPE
param_name = 'slope'
params = '/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/params_ch3_slope.gpkg'
params = gpd.read_file(params).groupby(['Class','ADM3_EN']).mean().reset_index()
params = params.loc[params['Class']=='Rain-fed Rice']
params = params.rename(columns=dict(zip(list(params.columns[2:20]), list(np.arange(2001, 2019, 1)))))
params['param']= param_name
slope = params.melt(id_vars=["Class", "ADM3_EN", "param"], var_name="Date", value_name='value')
slope['Date'] = 2017


In [7]:
# Subset temporally and pivot to table for linear regression
startYear = 2012
endYear = 2017
merged = pd.concat([ndvi, pcp, lstday, lstnight, et, pet, elevation, slope])
subbed = merged[(merged.Date <= endYear) & (merged.Date >= startYear)].groupby(['ADM3_EN','param']).mean().reset_index(drop=False)

df = subbed.pivot(index="ADM3_EN", columns=["param"],values="value").reset_index(drop=False)
'''df = df.loc[df['ADM3_EN'].isin([
'Carles',
'Calinog',
'Estancia',
'Lambunao',
'Cabatuan',
'Duenas',
'Santa Barbara',
#'Bingawan',
'Maasin',
'Badiangan'
])]'''


df

param,ADM3_EN,LST_Day_1km,LST_Night_1km,elevation,et,ndvi,pcp,pet,slope
0,Ajuy,28.687320,21.384192,70.885714,29.941365,2.131713,323.840146,49.947264,2.308457
1,Alimodian,30.453222,21.905168,161.947368,26.341044,2.076395,245.505945,50.085343,4.135536
2,Anilao,32.098078,21.749143,46.941176,23.960744,1.988011,236.849850,47.321698,2.535541
3,Badiangan,29.336703,21.551225,71.357143,27.894246,2.273580,282.728989,48.556002,1.676291
4,Balasan,29.561098,21.754047,21.777778,27.847374,2.012520,429.465698,49.916319,2.001327
5,Banate,31.512564,21.882481,34.352941,23.692293,1.973951,216.948468,46.791285,1.544544
6,Barotac Nuevo,31.749642,21.777504,7.960000,24.027006,1.869618,263.146109,48.764032,0.334543
7,Barotac Viejo,30.148663,21.568225,67.906250,28.394773,2.089180,277.032807,49.765890,2.789280
8,Batad,29.195222,21.549565,39.588235,27.791231,2.096498,414.478554,50.318205,3.641025
9,Bingawan,28.852831,21.123917,87.166667,26.819907,2.260217,336.040811,48.003781,1.189152


In [25]:
formula = 'ndvi ~ et + pet + elevation'
results = sm.ols(formula = formula, 
                 missing='drop',
                 data=df).fit()
print(results.summary())
print(pd.DataFrame(results.params).round(decimals=4).transpose())

                            OLS Regression Results                            
Dep. Variable:                   ndvi   R-squared:                       0.784
Model:                            OLS   Adj. R-squared:                  0.767
Method:                 Least Squares   F-statistic:                     47.15
Date:                Wed, 15 Dec 2021   Prob (F-statistic):           4.82e-13
Time:                        18:58:42   Log-Likelihood:                 59.787
No. Observations:                  43   AIC:                            -111.6
Df Residuals:                      39   BIC:                            -104.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.2355      0.263     12.322      0.0